
# 🏪 店舗別包括ダッシュボード v6.1 — 複数店舗選択・特徴量提案強化

- 指定CSVを読み込み、売上に効く特徴量を特定し、店舗選択に応じた提案を自動生成
- 在庫(on_hand)/原価(cost)/棚容量は未提供 → 依存分析は除外
- ダッシュボードは KPI / ABC / 特徴量 / 提案 / アラート を切替


## 店長の使い方（意思決定ガイド)
このダッシュボードは『見る→判断→即アクション』の順に作られています。毎日5〜10分で下記を実施してください。

### 1) KPI（全体傾向の把握：1分）
- チェック: 売上・客数・客単価の推移。昨日/一昨日と比べて大きく上下していないか。
- 行動: 下振れ時は次の『アラート』『提案』で原因と打ち手を確認し、該当カテゴリの面出し/フェース増や前出し時間の前倒しを実施。

### 2) アラート（急変の早期対応：1〜2分）
- 定義: 売上の3日移動平均と7日移動平均の乖離が±30%を超えた店舗日。
- 行動: 上振れ→該当カテゴリのフェース/在庫面（前出し）を増やす。下振れ→陳列場所や導線の見直し・値頃訴求（のちの見切り基準を調整）。

### 3) 提案（今日の打ち手：2〜3分）
- ロジック: 天候・曜日・イベント等のフラグごとに、売上が上振れしやすいカテゴリを提示。
- 行動: 上位カテゴリに対して、時間帯別の前出し/フェース増/導線強化を実行。例:
  - 降雨フラグ: 温かい総菜/カップ麺/ホット飲料/中華まん（入口側/レジ横の導線強化）
  - 猛暑日/真夏日: 冷飲料/アイス/チルドデザート（冷ケースのフェース増、氷/保冷品の訴求）
  - 週末: 弁当/麺/デザート（昼・夕ピーク前の前出し強化）
  - 給料日/給料日直後: 高単価弁当/スイーツ（夕方手前での厚め陳列）
  - 月初3日: 主力定番の面確保（発注強めの判断材料に）
  - 月末3日: 値頃品/節約訴求（価格ポップ/クロスMD）

### 4) ABC（棚の配分調整：2分）
- A: 売上80%を作る主力。フェースを最優先で確保し、欠品させない（前出し時間を前倒し）。
- B: 補完。ピーク前補充と見せ方で差を出す。
- C: 圧縮/入替候補。売場スペースをA/Bへ寄せる。

### 日次チェックリスト（例）
- [ ] KPIで下振れはないか→ある場合は『アラート』『提案』を優先確認
- [ ] 今日の提案キー（降雨/猛暑/週末/給料日など）と上位カテゴリをメモ
- [ ] 開店前: 上位カテゴリの面出し/フェース増（入口/レジ横/端導線）
- [ ] 昼ピーク前（〜11時）: 補充/前出し強化
- [ ] 夕ピーク前（〜17時）: 補充/前出し強化
- [ ] 閉店前: 値頃訴求/見切り（在庫連動は未実装のため判断基準を店舗運用で）

### 補足
- 本シートは在庫・原価なしの前提です。欠品/廃棄/粗利の最適化は別ノート（在庫連動）と連携を検討。
- 特徴量（要因）の上位を『明日』の打ち手（陳列/販促/導線）へ反映し、翌日のKPI改善に繋げてください。

In [ ]:
# ライブラリ
import warnings; warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
from pathlib import Path
import font_setup  # 日本語フォント/Plotlyテンプレート

try:
    import plotly.express as px; import plotly.graph_objects as go
    PLOTLY=True
except Exception:
    PLOTLY=False
try:
    import ipywidgets as widgets; from IPython.display import display, clear_output
    WIDGETS=True
except Exception:
    WIDGETS=False
try:
    from sklearn.ensemble import RandomForestRegressor
    SK_OK=True
except Exception:
    SK_OK=False

# ========================================
# 🚀 GPU検出と設定
# ========================================
print('='*60)
print('🖥️ GPU検出')
print('='*60)

GPU_AVAILABLE = False
GPU_DEVICE = 'cpu'

try:
    import torch
    if torch.cuda.is_available():
        GPU_AVAILABLE = True
        GPU_DEVICE = 'cuda'
        print(f'✅ NVIDIA GPU検出: {torch.cuda.get_device_name(0)}')
        print(f'   CUDA Version: {torch.version.cuda}')
        print(f'   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
    else:
        print('⚠️ PyTorch installed but no CUDA GPU found')
except ImportError:
    print('ℹ️ PyTorch not installed (GPU detection skipped)')


# cuDF (GPU-accelerated Pandas) チェック
CUDF_AVAILABLE = False
try:
    import cudf
    if GPU_AVAILABLE:
        CUDF_AVAILABLE = True
        print('✅ cuDF (GPU Pandas) 対応: 可能')
        print('   💡 大規模データ処理が10～50倍高速化されます')
    else:
        print('ℹ️ cuDF: GPU未検出のためスキップ')
except ImportError:
    print('ℹ️ cuDF not installed (pip install cudf-cu12)')

# GPU使用フラグ
USE_GPU = GPU_AVAILABLE

if USE_GPU:
    print(f'🚀 GPU使用: 有効（データ処理高速化）')
    print(f'   Device: {GPU_DEVICE}')
else:
    print(f'\nℹ️ GPU使用: 無効（CPUモードで実行）')

print('='*60)

print(f"\nEnv: pandas={pd.__version__}, plotly={'OK' if PLOTLY else 'N/A'}, widgets={'OK' if WIDGETS else 'N/A'}, sklearn={'OK' if SK_OK else 'N/A'}, GPU={'YES' if GPU_AVAILABLE else 'NO'}")

In [ ]:
# ========================================
# 🔧 GPU/CPU データフレーム変換ユーティリティ
# ========================================

def to_gpu(df):
    """pandasデータフレームをGPU (cuDF) に変換（USE_GPU=Trueの場合のみ）"""
    if USE_GPU and CUDF_AVAILABLE and df is not None and not df.empty:
        try:
            import cudf
            return cudf.from_pandas(df)
        except Exception as e:
            print(f'⚠️ GPU変換失敗、CPUモード継続: {e}')
            return df
    return df

def to_cpu(df):
    """cuDFデータフレームをpandasに変換"""
    if df is None:
        return None
    try:
        import cudf
        if isinstance(df, cudf.DataFrame):
            return df.to_pandas()
    except:
        pass
    return df

# GPUメモリ使用状況表示
def show_gpu_memory():
    if GPU_AVAILABLE:
        try:
            import torch
            allocated = torch.cuda.memory_allocated(0) / 1024**3
            reserved = torch.cuda.memory_reserved(0) / 1024**3
            print(f'📊 GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved')
        except:
            pass

if USE_GPU and CUDF_AVAILABLE:
    print('✅ GPU高速化ユーティリティ: 準備完了')
    print('   to_gpu(df) でGPU処理、to_cpu(df) でCPU戻し')
    show_gpu_memory()
else:
    print('ℹ️ CPU処理モード（GPUユーティリティは無効）')


## 1. 指定CSV読み込みと整形
- 生成: `sales`(必須), `product`(SKUマスタ), `df`(解析用ベース), `weather`(代表列)

In [ ]:

# データ読み込み（環境変数優先→既定パス）
import os
cand_env = [os.environ.get('DATA_PATH'), os.environ.get('ENRICHED_CSV')]
DATA_PATH = None
for p in cand_env:
    if p and Path(p).exists():
        DATA_PATH = Path(p); break
if DATA_PATH is None:
    DATA_PATH = Path('output/06_final_enriched_20250701_20250930.csv')

raw = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
df = raw.copy()
# 列名正規化（存在すれば）
rename_map = {
    '店舗':'store_id','商品名':'sku_id','日付':'date','売上数量':'qty','売上金額':'sales_amt',
    'フェイスくくり大分類':'category_l','フェイスくくり中分類':'category_m','フェイスくくり小分類':'category_s'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])
if 'qty' in df.columns: df['qty'] = pd.to_numeric(df['qty'], errors='coerce').fillna(0)
if 'sales_amt' in df.columns: df['sales_amt'] = pd.to_numeric(df['sales_amt'], errors='coerce').fillna(0)
if {'sales_amt','qty'}.issubset(df.columns): df['price'] = np.where(df['qty']>0, df['sales_amt']/df['qty'], np.nan)

sales = df[[c for c in ['date','store_id','sku_id','qty','price','sales_amt','category_l','category_m','category_s'] if c in df.columns]].copy()
product = df[[c for c in ['sku_id','category_l','category_m','category_s'] if c in df.columns]].drop_duplicates().copy() if 'sku_id' in df.columns else pd.DataFrame()

# weather（代表列があれば集約）
wcols = [c for c in ['天気','最高気温','最低気温','降水量','平均気温','気温差'] if c in df.columns]
if wcols and {'date','store_id'}.issubset(df.columns):
    wdf = df[['date','store_id']+wcols].copy()
    num_cols = [c for c in wcols if c!='天気']
    agg = {c:'mean' for c in num_cols}
    if '天気' in wcols:
        # 最頻値を代表とする
        def _mode(x):
            m = x.mode()
            return m.iloc[0] if not m.empty else x.iloc[0]
        agg['天気'] = _mode
    weather = wdf.groupby(['date','store_id'], as_index=False).agg(agg)
    if '天気' in weather.columns: weather = weather.rename(columns={'天気':'weather'})
else:
    weather = pd.DataFrame()

print('読み込み:', {'raw': raw.shape, 'sales': sales.shape, 'product': product.shape, 'weather': getattr(weather,'shape',None)})

## 2. KPI/ABC ユーティリティ

In [ ]:

def compute_daily_kpi(sales_df, prod_df):
    if sales_df is None or sales_df.empty:
        return pd.DataFrame()
    df0 = sales_df.copy()
    if 'sales_amt' not in df0.columns:
        if {'price','qty'}.issubset(df0.columns):
            df0['sales_amt'] = df0['price'] * df0['qty']
        else:
            df0['sales_amt'] = 0
    if not prod_df.empty and 'cost' in prod_df.columns:
        df0 = df0.merge(prod_df[['sku_id','cost']], on='sku_id', how='left')
        df0['gross_profit'] = df0['sales_amt'] - df0['cost'].fillna(0) * df0['qty'].fillna(0)
    agg_map = dict(sales_amt=('sales_amt','sum'), qty=('qty','sum'))
    if 'gross_profit' in df0.columns:
        agg_map['gross_profit'] = ('gross_profit','sum')
    gp = df0.groupby(['date','store_id'], as_index=False).agg(**agg_map)
    return gp

def abc_analysis(sales_df, prod_df, top_ratio=(0.8,0.95)):
    if sales_df is None or sales_df.empty:
        return pd.DataFrame()
    d = sales_df.copy()
    if 'sales_amt' not in d.columns:
        if {'price','qty'}.issubset(d.columns):
            d['sales_amt'] = d['price'] * d['qty']
        else:
            d['sales_amt'] = 0
    by = d.groupby('sku_id', as_index=False)['sales_amt'].sum().sort_values('sales_amt', ascending=False)
    if by['sales_amt'].sum() == 0:
        by['cum_ratio'] = 0
    else:
        by['cum_ratio'] = by['sales_amt'].cumsum()/by['sales_amt'].sum()
    a_th, b_th = top_ratio
    by['ABC'] = np.where(by['cum_ratio']<=a_th, 'A', np.where(by['cum_ratio']<=b_th, 'B','C'))
    if not prod_df.empty:
        by = by.merge(prod_df, on='sku_id', how='left')
    return by

## 3. 店舗選択（複数可）

In [ ]:
# 店舗選択ウィジェット（この時点では表示のみ、選択は cell-14 で行う）
store_opts = sorted(sales['store_id'].dropna().unique().tolist()) if not sales.empty and 'store_id' in sales.columns else []
if WIDGETS and store_opts:
    print(f'✅ 店舗選択ウィジェットを準備しました（{len(store_opts)}店舗）')
    print('👉 下の「6. ダッシュボード」セクションで店舗を選択してください')
else:
    print('⚠️ widgets無効、または店舗データなし')


## 4. 需要予測（PyCaret）
- Exogenous: 天候/カレンダー/プロモなど（本サンプルでは未結合）
- SKU×店舗の単一系列サンプル予測

In [ ]:
try:
    from pycaret.time_series import setup as ts_setup, compare_models as ts_compare, finalize_model as ts_finalize, predict_model as ts_predict
    PYCaret_TS=True
except Exception as e:
    PYCaret_TS=False
    print(f'PyCaret TS インポートエラー: {e}')

# PyCaret TSは cell-14 のダッシュボードで実行（店舗選択後）
if PYCaret_TS:
    print('✅ PyCaret TS 利用可能（ダッシュボードで実行されます）')
elif not PYCaret_TS:
    print('⚠️ PyCaret TS 無効（このセクションはスキップされます）')


## 5. 特徴量重要度（売上金額）と提案の自動生成
- 選択店舗の合算売上（売上金額）を目的変数に、提供特徴量から重要度を推定
- sklearnが無い場合は相関係数で代替
- 上位特徴量に応じてカテゴリ別の売上差分を算出し、実行提案を出力

In [ ]:
# 特徴量作成
def build_feature_dataset(df_all, stores):
    d = df_all.copy()
    if stores: d = d[d['store_id'].isin(stores)]
    cand = [
        '祝日フラグ','土曜フラグ','日曜フラグ','週末フラグ','平日フラグ','休日フラグ','降雨フラグ','弱雨','普通雨','強雨','豪雨',
        '最高気温','最低気温','平均気温','気温差','降水量','最高気温_t-1','最高気温_t-7','最低気温_t-1','最低気温_t-7','平均気温_t-1','平均気温_t-7',
        '最高気温_MA3','最高気温_MA7','平均気温_MA3','平均気温_MA7','気温トレンド_7d','季節変動指数_月','季節変動指数_週','週'
    ]
    present = [c for c in cand if c in d.columns]
    keep = ['date','store_id','sales_amt'] + present
    t = d[keep].copy()
    # 日付×店舗で特徴量を代表化（数値は平均）、売上は合計
    Xdf = t.groupby(['date','store_id'], as_index=False)[present].mean() if present else t[['date','store_id']].drop_duplicates()
    ydf = t.groupby(['date','store_id'], as_index=False)['sales_amt'].sum().rename(columns={'sales_amt':'y'})
    feat = Xdf.merge(ydf, on=['date','store_id'], how='inner')
    return feat, present

# 重要度
def rank_features(df_all, stores):
    feat, cols = build_feature_dataset(df_all, stores)
    if len(feat)<10 or not cols:
        return pd.DataFrame(columns=['feature','importance'])
    if SK_OK:
        try:
            model = RandomForestRegressor(n_estimators=300, random_state=42)
            X = feat[cols].fillna(0).values
            y = feat['y'].values
            model.fit(X, y)
            imp = model.feature_importances_
            return pd.DataFrame({'feature': cols, 'importance': imp}).sort_values('importance', ascending=False)
        except Exception:
            pass
    # fallback: 相関
    dfm = feat[cols+['y']].copy()
    cor = dfm.corr(numeric_only=True)['y'].abs().drop('y', errors='ignore').sort_values(ascending=False)
    return cor.reset_index().rename(columns={'index':'feature'})

# 提案（フラグ別uplift）
def uplift_by_flag(df_all, stores, flag_col, top_n=5):
    if flag_col not in df_all.columns: return pd.DataFrame()
    d = df_all.copy()
    if stores: d = d[d['store_id'].isin(stores)]
    if 'category_l' not in d.columns: return pd.DataFrame()
    base = d.groupby(['date','store_id','category_l'], as_index=False)['sales_amt'].sum()
    flags = d[['date','store_id','category_l', flag_col]].drop_duplicates()
    m = base.merge(flags, on=['date','store_id','category_l'], how='left')
    a = m[m[flag_col]==1].groupby('category_l', as_index=False)['sales_amt'].mean()
    b = m[m[flag_col]!=1].groupby('category_l', as_index=False)['sales_amt'].mean()
    if a.empty or b.empty: return pd.DataFrame()
    u = a.merge(b, on='category_l', suffixes=('_flag1','_flag0'))
    u['uplift'] = (u['sales_amt_flag1']-u['sales_amt_flag0'])/(u['sales_amt_flag0']+1e-6)
    return u.sort_values('uplift', ascending=False).head(top_n)

# 実行
fi = rank_features(df, selected_stores if 'selected_stores' in globals() else [])
display(fi.head(20))
print("\n--- 提案（カテゴリ別の売上上振れが大きい順）---")
for f in [c for c in ['降雨フラグ','週末フラグ','猛暑日','真夏日','夏日','冬日','真冬日','給料日','給料日直後','月初3日','月末3日'] if c in df.columns]:
    top_u = uplift_by_flag(df, selected_stores if 'selected_stores' in globals() else [], f, top_n=5)
    if not top_u.empty:
        print(f"[提案キー: {f}]")
        display(top_u)

In [ ]:
# ========================================
# 📉 Downlift分析：売上減少要因の特定
# ========================================

print('\n' + '='*80)
print('📉 Downlift分析：トリガー別の売上減少カテゴリ')
print('='*80)

# 売上列の確認（変換後の列名）
if 'sales_amt' in df.columns:
    sales_col = 'sales_amt'
elif '売上金額' in df.columns:
    sales_col = '売上金額'
elif '売上数量' in df.columns:
    sales_col = '売上数量'
else:
    print('⚠️ 売上列が見つかりません')
    sales_col = None

# カテゴリ列の確認
if 'category_l' in df.columns:
    category_col = 'category_l'
elif 'フェイスくくり大分類' in df.columns:
    category_col = 'フェイスくくり大分類'
elif 'category_m' in df.columns:
    category_col = 'category_m'
else:
    print('⚠️ カテゴリ列が見つかりません')
    category_col = None

print(f'📊 使用する売上列: {sales_col}')
print(f'📊 使用するカテゴリ列: {category_col}')

if sales_col is None or category_col is None:
    print('\n⚠️ 必要な列が見つからないため、Downlift分析をスキップします')
else:
    # 降雨時に売上が減少するカテゴリ（downlift）
    downlift_results = {}

    for trigger_col in ['降雨フラグ', '週末フラグ', '猛暑日', '真夏日', '夏日', '給料日', '給料日直後', '月初3日', '月末3日']:
        if trigger_col not in df.columns:
            continue

        # トリガーON/OFF時の売上比較（downlift: flag1 < flag0）
        try:
            comparison = df.groupby([trigger_col, category_col], as_index=False)[sales_col].mean()

            pivot = comparison.pivot_table(
                index=category_col,
                columns=trigger_col,
                values=sales_col,
                aggfunc='mean'
            )

            if 1.0 not in pivot.columns or 0.0 not in pivot.columns:
                continue

            # Downlift計算：(OFF - ON) / OFF （正の値 = 売上減少）
            pivot['downlift'] = (pivot[0.0] - pivot[1.0]) / (pivot[0.0] + 1e-6)

            # 売上減少が大きい順（downlift > 0.1 = 10%以上減少）
            significant_down = pivot[pivot['downlift'] > 0.1].copy()

            if len(significant_down) > 0:
                significant_down = significant_down.sort_values('downlift', ascending=False)
                significant_down = significant_down.rename(columns={
                    0.0: f'{sales_col}_flag0',
                    1.0: f'{sales_col}_flag1'
                })

                downlift_results[trigger_col] = significant_down[[f'{sales_col}_flag1', f'{sales_col}_flag0', 'downlift']].head(5)
        except Exception as e:
            print(f'⚠️ {trigger_col} の分析でエラー: {str(e)[:50]}')
            continue

    # ========================================
    # 結果表示
    # ========================================

    if downlift_results:
        print('\n--- 提案（カテゴリ別の売上減少が大きい順）---')
        for trigger_key, result_df in downlift_results.items():
            if len(result_df) > 0:
                print(f'\n[提案キー: {trigger_key}]')
                print(result_df.reset_index().to_string(index=False))

        print('\n' + '='*80)
        print('💡 Downlift活用方法')
        print('='*80)
        print('''
【在庫最適化】
- 降雨時に売上が減るカテゴリ → 雨予報の日は発注を控える
- 週末に売上が減るカテゴリ → 平日に在庫を厚くする

【キャンペーン企画】
- 売上減少カテゴリに対して「雨の日割引」「週末特売」などで需要喚起
- 例：降雨時10%減のカテゴリ → 雨の日10%割引で需要を維持

【販売計画】
- Downlift率を考慮した売上予測（保守的見積もり）
- 例：給料日直後-15%、月末-20%など季節変動を反映
        ''')
    else:
        print('\n⚠️ 有意なdownlift（売上減少10%以上）は検出されませんでした')

    # ========================================
    # 📊 Uplift vs Downliftの統合ビュー
    # ========================================

    print('\n' + '='*80)
    print('📊 統合分析：Uplift + Downlift')
    print('='*80)

    # カテゴリごとの売上変動幅を計算
    category_volatility = []

    for category in df[category_col].dropna().unique():
        cat_data = df[df[category_col] == category]

        max_uplifts = []
        max_downlifts = []

        for trigger_col in ['降雨フラグ', '週末フラグ', '猛暑日', '真夏日', '夏日']:
            if trigger_col not in df.columns:
                continue

            try:
                comparison = cat_data.groupby(trigger_col)[sales_col].mean()

                if 1.0 in comparison.index and 0.0 in comparison.index:
                    uplift = (comparison[1.0] - comparison[0.0]) / (comparison[0.0] + 1e-6)
                    downlift = (comparison[0.0] - comparison[1.0]) / (comparison[0.0] + 1e-6)

                    max_uplifts.append(max(0, uplift))
                    max_downlifts.append(max(0, downlift))
            except:
                continue

        if max_uplifts and max_downlifts:
            category_volatility.append({
                'カテゴリ': category,
                '最大Uplift': max(max_uplifts),
                '最大Downlift': max(max_downlifts),
                '売上変動幅': max(max_uplifts) + max(max_downlifts),
                '予測難易度': 'A:高難易度' if (max(max_uplifts) + max(max_downlifts)) > 1.5 else
                             'B:中難易度' if (max(max_uplifts) + max(max_downlifts)) > 0.5 else
                             'C:低難易度'
            })

    if category_volatility:
        volatility_df = pd.DataFrame(category_volatility).sort_values('売上変動幅', ascending=False)

        print('\n【カテゴリ別売上変動幅ランキング（Top 10）】')
        print(volatility_df.head(10).to_string(index=False))

        print('\n💡 予測モデリング推奨戦略:')
        print('  A:高難易度（変動幅1.5倍以上） → 個別モデル + 特徴量エンジニアリング強化')
        print('  B:中難易度（変動幅0.5-1.5倍） → カテゴリ別モデル')
        print('  C:低難易度（変動幅0.5倍未満） → 統合モデルでOK')

        # CSV保存
        output_path = Path('output/category_uplift_downlift_analysis.csv')
        volatility_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f'\n✅ 分析結果を保存: {output_path}')
    else:
        print('\n⚠️ カテゴリ別変動幅の計算に失敗しました')



## 6. ダッシュボード（店舗選択 + KPI/ABC/特徴量/提案/アラート）
- 店舗の複数選択に対応
- 在庫/原価依存のビューは除外（未提供のため）

In [ ]:
# 店舗複数選択 → 5ビューを同時展開（KPI/ABC/特徴量/提案/アラート）
if WIDGETS and (not sales.empty):
    store_opts = sorted(sales['store_id'].dropna().unique().tolist())
    stores_ms = widgets.SelectMultiple(
        options=store_opts, 
        value=tuple(store_opts[:1]) if store_opts else tuple(), 
        description='店舗(複数)',
        rows=min(10, len(store_opts))
    )
    out = widgets.Output()
    
    print('='*60)
    print('🏪 店舗を選択してください（複数選択可：Ctrl/Cmd + クリック）')
    print('='*60)
    display(stores_ms)
    display(out)

    def render_all(sel):
        with out:
            clear_output(wait=True)
            if not sel:
                print('⚠️ 店舗が選択されていません。上のリストから店舗を選択してください。')
                return
            
            print('='*60)
            print(f"📊 選択店舗: {', '.join(map(str, sel))}")
            print('='*60)
            
            ss = sales[sales['store_id'].isin(sel)].copy() if sel else sales.copy()
            
            # 1) KPI
            print('\n' + '='*60)
            print('[ビュー 1/5] KPI（日次売上）')
            print('='*60)
            print('📌 何を見る: 売上・客数・客単価の日次推移')
            print('📌 判断基準: 前日比で大きく下振れしていたら、次の「アラート」「提案」で原因を確認')
            print('-'*60)
            kpi = compute_daily_kpi(ss, product)
            if kpi is not None and not kpi.empty and PLOTLY:
                fig = px.line(kpi, x='date', y='sales_amt', color='store_id', title='売上金額(日次)')
                fig.show()
            else:
                display(kpi.head() if kpi is not None else 'KPIデータ無し')
            
            # 2) ABC
            print('\n' + '='*60)
            print('[ビュー 2/5] ABC（SKU寄与）')
            print('='*60)
            print('📌 何を見る: A（売上80%）、B（80-95%）、C（95-100%）のSKU分類')
            print('📌 判断基準: Aは絶対に欠品させない、Cは縮小・入替候補')
            print('-'*60)
            ab = abc_analysis(ss, product)
            if not ab.empty:
                print(f'✅ A: {len(ab[ab["ABC"]=="A"])} SKU, B: {len(ab[ab["ABC"]=="B"])} SKU, C: {len(ab[ab["ABC"]=="C"])} SKU')
                display(ab.head(30))
            else:
                print('データ無し')
            
            # 3) 特徴量重要度
            print('\n' + '='*60)
            print('[ビュー 3/5] 特徴量重要度（売上金額）')
            print('='*60)
            print('📌 何を見る: 売上に影響する要因のランキング（天候・曜日・イベント等）')
            print('📌 判断基準: 上位の要因を「提案」と組み合わせて、陳列・導線を調整')
            print('-'*60)
            fi = rank_features(df, sel)
            if not fi.empty:
                display(fi.head(20))
            else:
                print('データ不足（10行未満）のため、特徴量重要度は計算できません')
            
            # 4) 提案
            print('\n' + '='*60)
            print('[ビュー 4/5] 提案（カテゴリ別の売上上振れ候補）')
            print('='*60)
            print('📌 何を見る: 各フラグ（降雨/週末/猛暑/給料日等）で売上が伸びるカテゴリ')
            print('📌 判断基準: 今日該当するフラグの上位カテゴリを、開店前・ピーク前に前出し/フェース増')
            print('-'*60)
            flag_found = False
            for f in [c for c in ['降雨フラグ','週末フラグ','猛暑日','真夏日','夏日','冬日','真冬日','給料日','給料日直後','月初3日','月末3日'] if c in df.columns]:
                u = uplift_by_flag(df, sel, f, top_n=5)
                if not u.empty:
                    flag_found = True
                    print(f"\n🔑 [提案キー: {f}]")
                    display(u)
            if not flag_found:
                print('提案データなし（該当フラグ列が存在しない、またはデータ不足）')
            
            # 5) アラート
            print('\n' + '='*60)
            print('[ビュー 5/5] アラート（3日MA vs 7日MAの乖離）')
            print('='*60)
            print('📌 何を見る: 短期トレンド（3日）と中期トレンド（7日）の差が±30%を超えた日')
            print('📌 判断基準: 上振れ→該当カテゴリのフェース増、下振れ→陳列・導線・値頃訴求の見直し')
            print('-'*60)
            tmp = ss.groupby(['date','store_id'], as_index=False)['sales_amt'].sum().sort_values('date')
            if tmp.empty:
                print('データ無し')
            else:
                tmp['ma3'] = tmp.groupby('store_id')['sales_amt'].transform(lambda x: x.rolling(3, min_periods=1).mean())
                tmp['ma7'] = tmp.groupby('store_id')['sales_amt'].transform(lambda x: x.rolling(7, min_periods=1).mean())
                tmp['delta'] = (tmp['ma3'] - tmp['ma7'])/(tmp['ma7']+1e-6)
                alert = tmp[tmp['delta'].abs()>0.3].copy()
                if not alert.empty:
                    print(f'⚠️ アラート発生: {len(alert)} 件（最新20件を表示）')
                    display(alert.tail(20))
                else:
                    print('✅ アラートなし（乖離が30%未満）')
            
            print('\n' + '='*60)
            print('✅ 全5ビューの表示完了')
            print('='*60)

    def _on_change(change):
        if change['name']=='value':
            render_all(list(change['owner'].value))

    stores_ms.observe(_on_change, names='value')
    # 初回描画
    render_all(list(stores_ms.value))
else:
    print('⚠️ widgets未利用、またはデータ無しのため、ダッシュボード展開はスキップ')

In [ ]:
# ========================================
# 📊 包括的売上インパクト分析（全特徴量）
# ========================================

print('\n' + '='*80)
print('📊 包括的売上インパクト分析（格納率80%以上の全特徴量）')
print('='*80)

# 売上列の確認
sales_col = '売上金額' if '売上金額' in df.columns else '売上数量'
print(f'\n📊 売上指標: {sales_col}')

# ========================================
# データ品質分析
# ========================================

# 各列の格納率計算
completeness = {}
for col in df.columns:
    non_null_count = df[col].notna().sum()
    completeness[col] = non_null_count / len(df)

# 格納率80%以上のカラム抽出
high_quality_cols = [col for col, rate in completeness.items() if rate >= 0.8]

print(f'✅ 格納率80%以上のカラム: {len(high_quality_cols)}個')

# 分析対象カラム（除外列を除く）
exclude_cols = ['店舗', '商品名', '日付', sales_col, 'date', 'store_id', 'sku_id',
               'category_l', 'category_m', 'category_s',
               'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類']

analysis_cols = [col for col in high_quality_cols if col not in exclude_cols]
print(f'   分析対象カラム: {len(analysis_cols)}個')

# ========================================
# 売上インパクト計算
# ========================================

import warnings
warnings.filterwarnings('ignore')

impact_results = []

for col in analysis_cols:
    try:
        col_data = df[col].dropna()

        if len(col_data) < 100:
            continue

        # 数値型の場合
        if pd.api.types.is_numeric_dtype(col_data):
            unique_vals = col_data.unique()

            # バイナリフラグ（0/1）
            if len(unique_vals) == 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0}):
                flag_on = df[df[col] == 1][sales_col].mean()
                flag_off = df[df[col] == 0][sales_col].mean()

                if flag_off > 0:
                    impact = (flag_on - flag_off) / flag_off
                    impact_abs = flag_on - flag_off

                    impact_results.append({
                        '特徴量': col,
                        '分析タイプ': 'バイナリフラグ',
                        '売上_ON': flag_on,
                        '売上_OFF': flag_off,
                        'インパクト率': impact,
                        'インパクト絶対値': impact_abs,
                        'サンプル数_ON': (df[col] == 1).sum(),
                        'サンプル数_OFF': (df[col] == 0).sum()
                    })

            # 連続値（相関分析）
            elif len(unique_vals) > 10:
                correlation = df[[col, sales_col]].corr().iloc[0, 1]

                if not np.isnan(correlation):
                    q75 = col_data.quantile(0.75)
                    q25 = col_data.quantile(0.25)

                    sales_high = df[df[col] >= q75][sales_col].mean()
                    sales_low = df[df[col] <= q25][sales_col].mean()

                    if sales_low > 0:
                        impact = (sales_high - sales_low) / sales_low
                        impact_abs = sales_high - sales_low

                        impact_results.append({
                            '特徴量': col,
                            '分析タイプ': '連続値（Q75 vs Q25）',
                            '売上_高': sales_high,
                            '売上_低': sales_low,
                            'インパクト率': impact,
                            'インパクト絶対値': impact_abs,
                            '相関係数': correlation
                        })

            # カテゴリカル数値（3-10種類）
            elif 3 <= len(unique_vals) <= 10:
                category_sales = df.groupby(col)[sales_col].agg(['mean', 'count'])

                if len(category_sales) >= 2:
                    max_sales = category_sales['mean'].max()
                    min_sales = category_sales['mean'].min()

                    if min_sales > 0:
                        impact = (max_sales - min_sales) / min_sales
                        impact_abs = max_sales - min_sales

                        impact_results.append({
                            '特徴量': col,
                            '分析タイプ': 'カテゴリカル',
                            '売上_最大': max_sales,
                            '売上_最小': min_sales,
                            'インパクト率': impact,
                            'インパクト絶対値': impact_abs,
                            'カテゴリ数': len(unique_vals)
                        })

    except Exception:
        continue

print(f'\n✅ 分析完了: {len(impact_results)}個の特徴量を分析')

# ========================================
# 結果集計
# ========================================

if len(impact_results) > 0:
    impact_df = pd.DataFrame(impact_results)
    impact_df['インパクト率_絶対値'] = impact_df['インパクト率'].abs()
    impact_df = impact_df.sort_values('インパクト率_絶対値', ascending=False)

    print('\n' + '='*80)
    print('🏆 売上インパクトランキング Top 20')
    print('='*80)

    for idx, row in impact_df.head(20).iterrows():
        impact_sign = '+' if row['インパクト率'] > 0 else ''
        print(f"{row['特徴量']:30s} {impact_sign}{row['インパクト率']:.2%} "              f"({impact_sign}{row['インパクト絶対値']:,.0f}円) [{row['分析タイプ']}]")

    # 正のインパクトTop 5
    positive_impact = impact_df[impact_df['インパクト率'] > 0].head(5)
    print('\n' + '='*80)
    print('✅ 売上増加要因 Top 5')
    print('='*80)
    for idx, row in positive_impact.iterrows():
        print(f"  {row['特徴量']}: {row['インパクト率']:+.2%} ({row['インパクト絶対値']:+,.0f}円)")

    # 負のインパクトTop 5
    negative_impact = impact_df[impact_df['インパクト率'] < 0].sort_values('インパクト率').head(5)
    print('\n' + '='*80)
    print('⚠️ 売上減少要因 Top 5')
    print('='*80)
    for idx, row in negative_impact.iterrows():
        print(f"  {row['特徴量']}: {row['インパクト率']:+.2%} ({row['インパクト絶対値']:+,.0f}円)")

    print('\n' + '='*80)
    print('🎯 推奨アクション')
    print('='*80)
    print('''
【売上最大化施策】
1. 正のインパクト要因を最大化
   - 該当条件での在庫確保
   - プロモーション強化
   - 価格最適化

2. 負のインパクト要因を緩和
   - 売上減少時の特別施策
   - 代替商品の提案
   - 割引キャンペーン

3. 予測モデルへの活用
   - インパクト率上位の特徴量を重点使用
   - 特徴量エンジニアリングで相互作用項作成
   - カテゴリ別に重要特徴量を選定
    ''')

else:
    print('\n⚠️ インパクト分析結果が得られませんでした')
